### Things to do

- Load data into npz files
- Filter nonresponding electrodes
- Analyze blocked and shuffled data in NCM and Field L
- Analyze contrast data

In [1]:
#imports
import numpy as np
import matplotlib.pylab as plt
import pdb
import mdlab as mdl
import dnlab as dnl
import pandas as pd
import scipy
import scipy.spatial as ssp
import scipy.stats as sst
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier as knn
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split as split
import matplotlib.patches as mpatches
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [2]:
header, spikes = mdl.load_npz_data("D:\\2019_Human_Speech\\ncm\\blocked\\hv_blocked.npz", False)
block = mdl.SpikeData(header, spikes)
mua_block=block.mua_or_sua("mua")
sep_mua_block = dnl.separate_npz_data(mua_block)

In [3]:
split_block = dnl.separate_npz_data(block)

In [46]:
uno = sep_mua_block[0]

In [84]:
test.spikes.shape

(320L, 12000L)

In [66]:
test = sep_mua_block[0].get_kw_SpikeData(electrode = 1100)
test2 = sep_mua_block[0].get_kw_SpikeData(electrode = 1200)

In [49]:
abso, logi = uno.get_index(electrode = 1100)

In [50]:
a = uno.get_idx_SpikeData(abso, logi)

In [53]:
test.get_kw_SpikeData({electrode:1100})

NameError: name 'electrode' is not defined

In [112]:
spikes = []
header = pd.DataFrame()
spikes.append(test.spikes)
spikes.append(test2.spikes)
header=header.append(test.header)
header=header.append(test2.header)
a=mdl.SpikeData(header, spikes)
np.vstack(spikes).shape

(640L, 12000L)

In [26]:
# loop through spike data
# loop through electrodes
# check if electrode is responding
# if responding, add header and spike
# combine at the end
def res_filter(split_data, p_value= 0.05):
    res_header = pd.DataFrame()
    res_spikes = []
    res_split_data = []
    for spike_data in split_data:
        for ele in spike_data.electrodes:
            ele_spike_data = spike_data.get_kw_SpikeData(electrode=ele)
            respond = ele_spike_data.res_check()
            try:
                if respond["res"][0] < 0.05:
                    res_spikes.append(ele_spike_data.spikes)
                    res_header = res_header.append(ele_spike_data.header)
            except:
                pdb.set_trace()
    
                #absidx, logidx = spike_data.get_index(electrode=ele)
                
                #res_logidx = np.hstack((res_logidx, logidx))
                #res_absidx = np.hstack((res_absidx, absidx))
                #pdb.set_trace()
                #print ele
                #print res_logidx.shape
        #return res_logidx
        res_spikes = np.vstack(res_spikes).tolist()
        #return res_header
        new_spike_data = mdl.SpikeData(res_header, np.asarray(res_spikes))
        res_split_data.append(new_spike_data)
    return res_split_data
        
    


In [9]:
pls = res_filter(sep_mua_block)

In [25]:
pls[1].spikes.shape

AttributeError: 'list' object has no attribute 'shape'

In [ ]:
header, spikes = mdl.load_npz_data("D:\\2019_Human_Speech\\ncm\\blocked\\hv_blocked.npz", False)

In [ ]:
block = mdl.SpikeData(header, spikes)

In [158]:
mua_block=block.mua_or_sua("mua")

In [162]:
sep_mua_block = dnl.separate_npz_data(mua_block)

In [33]:
block = mdl.load_NpzData2SpikeData("D:\\2019_Human_Speech\\ncm\\blocked\\hv_blocked.npz", convert_birdid= False)

ValueError: need more than 2 values to unpack

In [38]:
theWord = ['David', 'is', 'not?', 'cool']
Fail = "Fail"

for x in theWord:
    print(theWord)
    if x == 'not?':
        block = mdl.load_npz_data("D:\\2019_Human_Speech\\ncm\\blocked\\hv_blocked.npz", convert_birdid= False)
        print(block)
    else:
        print(Fail)
        
        

['David', 'is', 'not?', 'cool']
Fail
['David', 'is', 'not?', 'cool']
Fail
['David', 'is', 'not?', 'cool']
(           birdid  electrode  trial  stim  starting_time  ending_time  \
0       6676008.0        900      1     1       1.252152     1.701653   
1       6676008.0        900      2     1       6.252212     6.701713   
2       6676008.0        900      3     1      11.252272    11.701773   
3       6676008.0        900      4     1      16.252332    16.701833   
4       6676008.0        900      5     1      21.252392    21.701894   
5       6676008.0        900      6     1      26.252452    26.701953   
6       6676008.0        900      7     1      31.252512    31.702013   
7       6676008.0        900      8     1      36.252572    36.702073   
8       6676008.0        900      9     1      41.252632    41.702133   
9       6676008.0        900     10     1      46.252692    46.702193   
10      6676008.0        900     11     1      51.252752    51.702253   
11      6676008.0